In [1]:
from rdkit.Chem.rdmolfiles import MolFromPDBFile
from rdkit.Chem.rdchem import Mol
from rdkit.Chem import AllChem

import numpy as np
import rdkit.Chem as Chem
from rdkit.Chem import AddHs, AssignStereochemistry, HybridizationType, ChiralType, BondStereo, MolFromMol2File
from rdkit.Chem.AllChem import ComputeGasteigerCharges
import os
import sys
sys.path.append("../../")
from src.data.pocket_utils import get_atom_coordinates, find_pocket_atoms_RDKit
from src.utils.constants import ES_THRESHOLD, METAL_OX_STATES
from tqdm import tqdm 

In [2]:
from src.data.utils import pdb_to_rdkit_mol, mol2_to_rdkit_mol, get_vdw_radius, add_charges_to_molecule, get_node_features, get_edge_features, extract_charges_from_mol2, ionic_interaction_using_distance

In [3]:
def rdkit_mol_featurizer(mol: Mol, 
                         pairwise_function: callable,
                         ) -> tuple: 
    """
    Extracts graph features from a given RDKit molecule object.
    Parameters:
        mol (rdkit.Chem.rdchem.Mol): RDKit molecule object.
        protein_or_ligand_ids (list): A list of ints describing whether each atom in the molecule is a protein (-1) or a ligand (1) atom. 
    Returns:
        node_features (np.ndarray): A 2D array of shape (num_atoms, num_node_features) containing node features:
            (protein_or_ligand_id, atomic_num, atomic_mass, aromatic_indicator, ring_indicator, hybridization, chirality, num_heteroatoms, degree, num_hydrogens, partial_charge, formal_charge, num_radical_electrons)
        edge_features (np.ndarray): A 2D array of shape (num_bonds, num_edge_features) containing edge features.
        edge_indices (np.ndarray): A 2D array of shape (num_bonds, 2) containing the indices of the atoms connected by each bond.
    """

    AssignStereochemistry(mol, force=True, cleanIt=True)

    node_features = get_node_features(mol)
    
    edge_features, edge_indices = get_edge_features(mol, pairwise_function)

    return np.array(node_features, dtype='float64'), np.array(edge_features, dtype='float64'), np.array(edge_indices, dtype='int64')

In [4]:
# Define test cases
test_cases = [("CCCCCCCCCC=C", 0, 10), # expected output: None
              ("C=C", 0, 1), # expected output:  (0, 0, 0, 0, 0, 1.3261131312034182)
              ("CC(=O)O", 2, 3) # expected output: (0, 0, 0, 0, 0, 2.278544145756904)
              ]

def tag_atoms(mol, tag):
    for atom in mol.GetAtoms():
        atom.SetProp('protein_or_ligand', tag)

# Initialize RDKit molecules and conformers
molecules = []
for smiles, a1, a2 in test_cases:
    mol = Chem.MolFromSmiles(smiles)
    AllChem.EmbedMolecule(mol)
    molecules.append((mol, a1, a2))

#tag all of the atoms to be ligands in teh test case
for mol, a1, a2 in molecules:
    tag_atoms(mol, 'ligand')


[09:32:48] Molecule does not have explicit Hs. Consider calling AddHs()
[09:32:48] Molecule does not have explicit Hs. Consider calling AddHs()
[09:32:48] Molecule does not have explicit Hs. Consider calling AddHs()


In [5]:
from src.data.utils import vdw_interactions, ionic_interaction_using_distance, covalent_bonds

#for each molecule in the test case, get the node features and edge features for the different pairwise functions

for mol, a1, a2 in molecules:
    print("pairwise function: vdw_interactions")
    print("Molecule: ", Chem.MolToSmiles(mol))
    print("Node features: ")
    print(rdkit_mol_featurizer(mol, vdw_interactions)[0])
    print("Edge features: ")
    print(rdkit_mol_featurizer(mol, vdw_interactions)[1])
    print("Edge indices: ")
    print(rdkit_mol_featurizer(mol, vdw_interactions)[2])
    print("\n")

    print("pairwise function: ionic_interaction_using_distance")
    print("Molecule: ", Chem.MolToSmiles(mol))
    print("Node features: ")
    print(rdkit_mol_featurizer(mol, ionic_interaction_using_distance)[0])
    print("Edge features: ")
    print(rdkit_mol_featurizer(mol, ionic_interaction_using_distance)[1])
    print("Edge indices: ")
    print(rdkit_mol_featurizer(mol, ionic_interaction_using_distance)[2])
    print("\n")

    print("pairwise function: covalent_bonds")
    print("Molecule: ", Chem.MolToSmiles(mol))
    print("Node features: ")
    print(rdkit_mol_featurizer(mol, covalent_bonds)[0])
    print("Edge features: ")
    print(rdkit_mol_featurizer(mol, covalent_bonds)[1])
    print("Edge indices: ")
    print(rdkit_mol_featurizer(mol, covalent_bonds)[2])
    print("\n")

pairwise function: vdw_interactions
Molecule:  C=CCCCCCCCCC
Node features: 


0it [00:00, ?it/s]
0it [00:00, ?it/s]

11it [00:00, 871.63it/s]
11it [00:00, 1284.88it/s]
11it [00:00, 1587.06it/s]
11it [00:00, 1854.32it/s]
11it [00:00, 2458.69it/s]
11it [00:00, 2704.57it/s]
11it [00:00, 3580.70it/s]
11it [00:00, 4753.49it/s]
11it [00:00, 6913.00it/s]
11it [00:00, 13451.12it/s]
11it [00:00, 262144.00it/s]
11it [00:00, 164.89it/s]


[[ 1.     6.    12.011  0.     0.     3.     0.     0.     1.     0.
   0.     0.   ]
 [ 1.     6.    12.011  0.     0.     3.     0.     0.     2.     0.
   0.     0.   ]
 [ 1.     6.    12.011  0.     0.     3.     0.     0.     2.     0.
   0.     0.   ]
 [ 1.     6.    12.011  0.     0.     3.     0.     0.     2.     0.
   0.     0.   ]
 [ 1.     6.    12.011  0.     0.     3.     0.     0.     2.     0.
   0.     0.   ]
 [ 1.     6.    12.011  0.     0.     3.     0.     0.     2.     0.
   0.     0.   ]
 [ 1.     6.    12.011  0.     0.     3.     0.     0.     2.     0.
   0.     0.   ]
 [ 1.     6.    12.011  0.     0.     3.     0.     0.     2.     0.
   0.     0.   ]
 [ 1.     6.    12.011  0.     0.     3.     0.     0.     2.     0.
   0.     0.   ]
 [ 1.     6.    12.011  0.     0.     2.     0.     0.     2.     0.
   0.     0.   ]
 [ 1.     6.    12.011  0.     0.     2.     0.     0.     1.     0.
   0.     0.   ]]
Edge features: 


11it [00:00, 1224.65it/s]
11it [00:00, 1420.13it/s]
11it [00:00, 1814.86it/s]
11it [00:00, 1869.73it/s]
11it [00:00, 2428.79it/s]
11it [00:00, 2939.62it/s]
11it [00:00, 3557.51it/s]
11it [00:00, 4380.68it/s]
11it [00:00, 6567.59it/s]
11it [00:00, 13646.06it/s]
11it [00:00, 316009.21it/s]
11it [00:00, 187.47it/s]


...

[[ 1.     6.    12.011  0.     0.     3.     0.     0.     1.     0.
   0.     0.   ]
 [ 1.     6.    12.011  0.     0.     2.     0.     2.     3.     0.
   0.     0.   ]
 [ 1.     8.    15.999  0.     0.     2.     0.     1.     1.     0.
   0.     0.   ]
 [ 1.     8.    15.999  0.     0.     2.     0.     1.     1.     0.
   0.     0.   ]]
Edge features: 


4it [00:00, 125203.10it/s]
4it [00:00, 90687.65it/s]
4it [00:00, 122461.43it/s]
4it [00:00, 4993.22it/s]
4it [00:00, 278.52it/s]


[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [2. 0. 1. 0. 0.]
 [2. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0.]]
Edge indices: 


4it [00:00, 102927.71it/s]
4it [00:00, 99864.38it/s]
4it [00:00, 5419.00it/s]
4it [00:00, 111107.39it/s]
4it [00:00, 239.89it/s]

[[0 1]
 [1 0]
 [1 2]
 [2 1]
 [1 3]
 [3 1]]


